In [ ]:
#####
# Initial cell
#####
import ipywidgets as widgets
from IPython.display import display
from collections.abc import Callable
from typing import Any


def multiple_choice[OptionType](question: str, 
                    options: list[OptionType], 
                    correct_option: OptionType, 
                    description: str = "") -> widgets.Widget:
    """
    Multiple-choice-single-answer type question.

    Delegates to generic_question.
    """
    options_widget = widgets.ToggleButtons(
        options=options,
        value=None,
        disabled=False
    )

    def eval_func(widget):
        if widget.value is None:
            return None
        return widget.value == correct_option

    return generic_question(question=question,
                            input_widget=options_widget,
                            evaluation_function=eval_func,
                            description=description)


def multiple_answers[OptionType](question: str, 
                                 options: list[OptionType], 
                                 correct_answers: list[OptionType]) -> widgets.Widget:
    """
    Multiple-choice-multiple-answers type question.

    Delegates to generic_question.

    """
    buttons = [widgets.ToggleButton(
        value=False, description=option) for option in options]

    def feedback(evaluation_result):
        if evaluation_result == None:
            return "Please pick an answer"
        elif evaluation_result == 0:
            return "Correct answer"
        else:
            return f"Correct answers: {evaluation_result}/{len(correct_answers)}"

    def eval_func(widget: widgets.HBox):
        answers = set(
            button.description for button in widget.children if button.value)
        if len(answers) == 0:
            return None
        # Evaluates number of correct choices minus number of incorrect choices.
        return len(answers.intersection(correct_answers)) - len(answers.difference(correct_answers))

    return generic_question(question=question,
                            input_widget=widgets.HBox(buttons),
                            evaluation_function=eval_func,
                            feedback=feedback)


def standard_feedback(evaluation_result: Any):
    if evaluation_result == None:
        return "No answer selected"
    elif evaluation_result == 0:
        return "Wrong answer!"
    else:
        return "Correct!"


def generic_question[Evaluation](question: str,
                     input_widget: widgets.Widget,
                     evaluation_function: Callable[[widgets.Widget], Evaluation],
                     description: str = "",
                     feedback: Callable[[Evaluation], str] = standard_feedback) -> widgets.Widget:
    """
    Abstract question function used by the other question types to display questions.

    Delegates to generic_question.

    params:
    - question: Title of question
    - input_widget: Widget used for getting user input
    - evaluation_function: a function returning an evaluation of the answer provided based on the input widget
    - description: Additional text to be provided with the question
    - feedback: A function giving textual feedback based on the result of the evaluation_function

    """

    title_widget = widgets.HTMLMath(value=f"<h3>{question}</h3>")
    description_widget = widgets.HTMLMath(value=f"<p>{description}</p>")

    output = widgets.Output()

    def _inner_check(button):
        with output:
            output.outputs = [
                {'name': 'stdout', 'text': feedback(evaluation_function(input_widget)), 'output_type': 'stream'}]

    button = widgets.Button(description="Check answer", icon="check",
                            style=dict(
                                button_color="lightgreen"
                            ))
    button.on_click(_inner_check)

    layout = widgets.VBox([title_widget,
                           description_widget,
                           widgets.HBox([input_widget],
                                        layout=widgets.Layout(padding="10px 20px 10px 20px", border="solid")),
                           widgets.VBox([button, output],
                                        layout=widgets.Layout(margin="10px 10px 0px 0px"))])

    return layout


def numeric_input(question: str, correct_answer: float) -> widgets.Widget:
    """
    Question with box for numeric input.

    Delegates to generic_question.
    """

    input_widget = widgets.FloatText(
        value=None,
    )

    def eval_func(widget):
        if widget.value is None:
            return None
        return widget.value == correct_answer

    return generic_question(question=question,
                            input_widget=input_widget,
                            evaluation_function=eval_func)


def code_question(question: str, expected_outputs: list[tuple[tuple, Any]]) -> widgets.Widget:
    """
    Code question that uses a textbox for the user to write.
    The provided function is tested against the expected_outputs.

    Delegates to generic_question.

    params:
    - expected_output - a list of pairs in the format:
        - ((inputs), expected_output)
        - Example: [
            ((2, 4), 8)
        ]
    """

    input_widget = widgets.Text(
        description="What is the name of your function?", placeholder="myFunction",
        style=dict(description_width="initial"))

    def eval_func(widget):
        function_name = widget.value
        if function_name not in globals():
            # Error handling
            return None

        function = globals()[function_name]
        return all([function(*test_input) == test_output
                    for test_input, test_output in expected_outputs])

    def feedback(evaluation_result):
        if evaluation_result is None:
            return "No function defined with that name. Remember to run the cell to define the function."
        if evaluation_result:
            return "Correct!"
        else:
            return "Incorrect answer!"

    return generic_question(question=question, input_widget=input_widget, evaluation_function=eval_func, feedback=feedback)


def display_json() -> widgets.Widget:
    """
    Displays a list of questions based on the provided json-string.

    Delegates to the other questions functions based on question type.
    """

    pass

# Test quiz

In [ ]:
display(widgets.HTML("<h2>Test your knowledge</h2>"))

display(multiple_choice("What is your name?", ["Jakob", "Jørgen", "Ravi"], "Jakob", description="Choose the answer corresponding to your name."))

display(numeric_input("What is 2 + 5?", 7))

display(numeric_input("What is 2/4 + 1/4?", 0.75))

display(multiple_answers("Which of these are cheeses?",
                         options=["Gouda", "Emmentaler", "Brie", "Cat", "Dog"],
                         correct_answers=["Gouda", "Emmentaler", "Brie"]))



HTML(value='<h2>Test your knowledge</h2>')

In [ ]:
# Define your function for coding task here:

def test(x, y):
    return 2+x+y

def tet(x, y):
    return x+y

In [ ]:

display(code_question(
    "Make a function that returns the sum of two numbers", [((2, 2), 4)]))

In [58]:
# # title = widgets.Text(value="What is the best pizza topping")
# title = widgets.HTML(value="<h2>What is the best topping?</h2>")
# buttons = widgets.RadioButtons(
#     options=['pepperoni', 'pineapple', 'anchovies'],
#    value=None, # Defaults to 'pineapple'
# #    layout={'width': 'max-content'}, # If the items' names are long
#     disabled=False
# )

# from urllib.request import urlopen

# # file = urlopen("https://plus.unsplash.com/premium_photo-1664474619075-644dd191935f?fm=jpg&q=60&w=3000&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8aW1hZ2V8ZW58MHx8MHx8fDA%3D")
# file = open("../figures/image.jpg", "rb")
# image = file.read()
# image = widgets.Image(
#     value=image,
#     format='jpg',
#     width=300,
#     height=400,
# )

# def f(value):
#     if value == None:
#         print("Answer!")
#     # print(buttons)
#     # value = buttons.value
#     elif value == "pepperoni":
#         print("Correct!")
#     else:
#         print("Incorrect!")

# display(title)
# display(image)
# display(buttons)
# display(widgets.interactive_output(f, {"value": buttons}))

# output = widgets.Output()
# button = widgets.Button(description="Check answer")
# def _inner_check(button):
#     with output:
#         if "test" not in globals():
#             output.outputs = [{'name': 'stdout', 'text': 'Please define the function in the cell above!', 'output_type': 'stream'}]
#         elif test(2) == 4:
#             output.outputs = [{'name': 'stdout', 'text': 'Correct!', 'output_type': 'stream'}]
#         else:
#             output.outputs = [{'name': 'stdout', 'text': 'Incorrect!', 'output_type': 'stream'}]
# button.on_click(_inner_check)
# display(button, output)
